In [1]:
import re
import csv
import time
import nltk
import json
import string
import numpy as np
import pandas as pd
from nltk import stem
import sklearn.metrics
from random import randint
from numpy.linalg import norm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

with open('../../new_userDict.json', 'r') as fp:
    
    new_userDict = json.load(fp)
    
with open('../../new_isbnDict.json', 'r') as fp:
    
    new_isbnDict = json.load(fp)
    
with open('../../dict_row.json', 'r') as fp:
    
    dict_row = json.load(fp)
    
with open('../../dict_col.json', 'r') as fp:
    
    dict_col = json.load(fp)
    
        
print("Ok")

Ok


In [2]:
with open('../../clustering/CLUSTERS_ITEMS.json', 'r') as fp:
    
    CLUSTERS_ITEMS = json.load(fp)
    
    
with open('../../clustering/CLUSTERS_USERS.json', 'r') as fp:
    
    CLUSTERS_USERS = json.load(fp)
    
    
with open('../../clustering/file/clusters_dict_row.json', 'r') as fp:
    
    clusters_dict_row = json.load(fp)
    
    
with open('../../clustering/file/clusters_dict_col.json', 'r') as fp:
    
    clusters_dict_col = json.load(fp)
    
    
with open('../../clustering/cluster_book_dict.json', 'r') as fp:
    
    cluster_book_dict = json.load(fp)
    
    
with open('../../clustering/cluster_users_dict.json', 'r') as fp:
    
    cluster_users_dict = json.load(fp)
    
print("Ok") 



with open('../../clustering/index_book_user_clusters/index_user_cluster.json', 'r') as fp:
    
    index_user_cluster = json.load(fp)
    
    
with open('../../clustering/index_book_user_clusters/index_book_cluster.json', 'r') as fp:
    
    index_book_cluster = json.load(fp)
    
print("Ok")

Ok
Ok


In [ ]:
# I take books inside the cluster book and also rated by user inside user cluster

In [3]:
def convert(user_number,book_number, index_user_cluster, index_book_cluster):
    
    user_cluster = index_user_cluster[str(user_number)]
    book_cluster = index_book_cluster[str(book_number)]
    
    return user_cluster, book_cluster

In [4]:
def createSampleDict(new_isbnDict, new_userDict, t1, t2):
    

    small_isbnDict = {}

    for book in new_isbnDict:

        temp = new_isbnDict[book]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t1:

            small_isbnDict[book] = new_isbnDict[book]   

    small_userDict = {}

    for user in new_userDict:

        temp = new_userDict[user]
        i = 0

        for t in temp.values():
            if t != "0":
                i = i+1

        if i >t2:

            small_userDict[user] = new_userDict[user]
            
            
    return small_isbnDict, small_userDict


def computeMatrices(train_userDict,train_isbnDict,small_userDict,small_isbnDict, dict_row, dict_col):

    n = len(small_isbnDict)
    m = len(small_userDict)
    
    index = sorted(small_userDict.keys())
    columns = sorted(small_isbnDict.keys())

    dict_row = {k:v for v,k in enumerate(index)}
    dict_col = {k:v for v,k in enumerate(columns)}

    u = np.zeros((m,n)) 
    R = np.zeros((m,n))
    for user in train_userDict:
        for isbn in train_userDict[user]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_userDict[user][isbn]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue

    for isbn in train_isbnDict:
        for user in train_isbnDict[isbn]:
            try:
                u[dict_row[user]][dict_col[isbn]] = train_isbnDict[isbn][user]
                R[dict_row[user]][dict_col[isbn]] = 1
            except:
                continue
                
    small_utility_DataFrame = pd.DataFrame(u, index = index, columns = columns)
    R = pd.DataFrame(R, index = index, columns = columns)

    return u, R, small_utility_DataFrame


def selectSample(i,b,list_user,small_userDict, small_isbnDict):

    test_index = [k for k in range((i*b),(i+1)*b)]
    train_index = [k for k in range(len(list_user)) if k not in test_index]

    train = list_user[train_index]
    test  = list_user[test_index]

    train_userDict = {}
    train_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in train:

        train_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                train_userDict[user]
                train_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
                
    test_userDict = {}
    test_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in test:

        test_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                test_userDict[user]
                test_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
    return train_userDict, train_isbnDict, test_userDict, test_isbnDict, test


#indices = [i for i in range(len(list_user))]
#indices = np.array(indices)
#k = 5
#b = len(small_userDict)//k


def selectSampleRandom(indices,b,list_user,small_userDict, small_isbnDict):
    
    test_index = np.random.choice(indices, size = b, replace=False)
    
    indices = np.delete(indices,test_index)
    
    train_index = [k for k in range(len(list_user)) if k not in test_index]
    

    train = list_user[train_index]
    test  = list_user[test_index]

    train_userDict = {}
    train_isbnDict = {isbn:{} for isbn in small_isbnDict}

    for user in train:

        train_userDict[user] = small_userDict[user]

    for isbn in small_isbnDict:
        for user in small_isbnDict[isbn]:

            try:
                train_userDict[user]
                train_isbnDict[isbn][user] = small_isbnDict[isbn][user]
            except:
                continue
                
    return train_userDict, train_isbnDict, test_userDict, test_isbnDict, test, indices

In [6]:
# different approach

In [5]:
def mainClusteringRMSE(utility_DataFrame, utility_DataFrame_clusters,R_clusters, 
                                 new_userDict, new_isbnDict,
                                 index_user_cluster, index_book_cluster,zeros = "n"):
    

    rmse_dict = {j:{} for j in new_userDict.keys()}

    for user in new_userDict:

        for book in new_userDict[user]:

            if new_userDict[user][book] != "0":

                try:
                    new_isbnDict[book][user]
                    rmse_dict[user][book] = new_userDict[user][book]
                except:
                    continue
                    
            if new_userDict[user][book] == "0" and zeros != "n":
                try:
                    new_isbnDict[book]
                    rmse_dict[user][book] = "1"
                except:
                    continue

    d = rmse_dict.keys()
    v = []

    for key in d:

        if rmse_dict[key] == {}:

            v.append(key)

    for i in v:

        del rmse_dict[i]

    rmse_vector = []
    
    for user in rmse_dict:

        for isbn in rmse_dict[user]:
            
            prediction_score = None
            user_number = user
            book_number = isbn
            
            try:
                user_cluster, book_cluster = convert(user_number,book_number, index_user_cluster, index_book_cluster)
                
                if R_clusters[book_cluster][user_cluster] == 1.0:
                    
                    prediction_score = utility_DataFrame_clusters[book_cluster][user_cluster]
                    true_score = utility_DataFrame[book_number][user_number]
                    
                if prediction_score == None:
                    
                    term1 = utility_DataFrame_clusters[book_cluster]
                    r1 = R_clusters[book_cluster]
                    term1 = term1[r1 ==1]
                    
                    term2 = utility_DataFrame_clusters.loc[user_cluster]
                    r2 = R_clusters.loc[user_cluster]
                    term2 = term2[r2==1]
                    
                    prediction_score = (np.mean(term1) + np.mean(term2))/2
            
            except:
                prediction_score = None
                
            if prediction_score != None:    
                
                rmse_vector.append(tuple([user_number, book_number, prediction_score, true_score]))


    rmse = np.sqrt((np.array([u[2]-u[3] for u in rmse_vector])**2).sum()/len(rmse_vector))
    
    return rmse, rmse_vector

In [17]:
u_clusters, R_clusters, utility_DataFrame_clusters = computeMatrices(CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                           CLUSTERS_USERS,CLUSTERS_ITEMS,
                                                                            clusters_dict_row,clusters_dict_col)

u, R, utility_DataFrame = computeMatrices(new_userDict,new_isbnDict,new_userDict,new_isbnDict,dict_row,dict_col)

In [22]:
rmse_test, rmse_vector = mainClusteringRMSE(utility_DataFrame, utility_DataFrame_clusters,R_clusters,
                                                                new_userDict, new_isbnDict,
                                                                index_user_cluster, index_book_cluster)

In [23]:
rmse_test

1.4098341175566782

In [24]:
rmse_vector[:100]

[('268171', '0440218039', 9.6666666666666661, 9.0),
 ('268171', '0425183394', 8.5, 8.0),
 ('268171', '0804115435', 6.0, 6.0),
 ('268171', '080411160X', 8.5, 8.0),
 ('268171', '055326981X', 7.0, 7.0),
 ('268171', '0064472051', 10.0, 10.0),
 ('268171', '0804115419', 4.0, 4.0),
 ('268171', '0515095826', 5.0, 5.0),
 ('268171', '0441004016', 8.5, 9.0),
 ('231560', '0446601241', 5.0, 5.0),
 ('231560', '0451452011', 7.0, 7.0),
 ('231560', '0380001411', 7.0, 7.0),
 ('231560', '0061091316', 8.0, 8.0),
 ('231560', '0671463004', 6.5, 5.0),
 ('231560', '0345434803', 8.4000000000000004, 9.0),
 ('231560', '0061012491', 8.0, 8.0),
 ('231560', '0345313860', 7.1666666666649999, 9.0),
 ('265595', '0140444254', 7.0, 7.0),
 ('265595', '1853260495', 8.0, 8.0),
 ('265595', '0749305401', 8.0, 8.0),
 ('265595', '1853260363', 7.0, 7.0),
 ('265595', '0486264653', 9.0, 9.0),
 ('162641', '0446530743', 7.0, 7.0),
 ('162641', '0312272057', 8.0, 8.0),
 ('162641', '038550926X', 9.0, 9.0),
 ('162641', '0679419454', 6.